In [2]:
import pandas as pd
import keras
from keras.models import Sequential,Model
from keras.layers import Dense, Dropout, BatchNormalization,Input
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import numpy as np
# Library for Gaussian process
# import GPy
##Library for visualization
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import matplotlib;matplotlib.rcParams['figure.figsize'] = (15,15)
import pylab 
import time
from sklearn.model_selection import train_test_split

In [3]:
df_loc = pd.read_csv("2D_biv_matern_large.csv", sep = ",")

N = len(df_loc)
s = np.vstack((df_loc["x"],df_loc["y"])).T
y = np.array(df_loc[["var1","var2"]])

### Basis functions

num_basis = [7**2,9**2,11**2]
knots_1d = [np.linspace(0,1,int(np.sqrt(i))) for i in num_basis]
#knots_1d = [np.linspace(0,1,i) for i in num_basis]
##Wendland kernel
K = 0
phi = np.zeros((N, sum(num_basis)))
for res in range(len(num_basis)):
    theta = 1/np.sqrt(num_basis[res])*2.5
    knots_s1, knots_s2 = np.meshgrid(knots_1d[res],knots_1d[res])
    knots = np.column_stack((knots_s1.flatten(),knots_s2.flatten()))
    for i in range(num_basis[res]):
        d = np.linalg.norm(s-knots[i,:],axis=1)/theta
        for j in range(len(d)):
            if d[j] >= 0 and d[j] <= 1:
                phi[j,i + K] = (1-d[j])**6 * (35 * d[j]**2 + 18 * d[j] + 3)/3
            else:
                phi[j,i + K] = 0
    K = K + num_basis[res]

In [66]:
# model layers 

input_dim = Input(shape = (phi.shape[1], ))
layer1 = Dense(100, kernel_initializer='he_uniform', activation = 'relu')(input_dim)
layer2 = Dense(100, activation = 'relu')(layer1)
layer3 = Dense(100, activation = 'relu')(layer2)
layer4 = Dense(100, activation = 'relu')(layer3)
layer5 = Dense(100, activation = 'relu')(layer4)
final_layer = Dense(2, activation = 'linear')(layer5)

In [67]:
s_train, s_test, encoder_train, encoder_test    , y_train, y_test= train_test_split(s, phi, y, 
                                                                                    test_size=0.1)
print(s_test.shape)

(640, 2)


In [68]:
base_model = Model(inputs = input_dim, outputs = final_layer)
optimizer = keras.optimizers.Adam(lr=0.01)
# Compile the Model
base_model.compile(optimizer = optimizer, loss = 'mse')
base_model.summary()
base_model.fit(encoder_train, y_train,validation_split = 0.1, epochs = 400,
                batch_size = 64,verbose = 2)

Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 251)               0         
_________________________________________________________________
dense_22 (Dense)             (None, 100)               25200     
_________________________________________________________________
dense_23 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_24 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_25 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_26 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_27 (Dense)             (None, 2)                 202

 - 0s - loss: 0.0497 - val_loss: 0.0799
Epoch 131/400
 - 0s - loss: 0.0492 - val_loss: 0.0829
Epoch 132/400
 - 0s - loss: 0.0466 - val_loss: 0.0811
Epoch 133/400
 - 0s - loss: 0.0496 - val_loss: 0.0822
Epoch 134/400
 - 0s - loss: 0.0508 - val_loss: 0.0821
Epoch 135/400
 - 0s - loss: 0.0488 - val_loss: 0.0829
Epoch 136/400
 - 0s - loss: 0.0477 - val_loss: 0.0840
Epoch 137/400
 - 0s - loss: 0.0474 - val_loss: 0.0854
Epoch 138/400
 - 0s - loss: 0.0483 - val_loss: 0.0830
Epoch 139/400
 - 0s - loss: 0.0474 - val_loss: 0.0871
Epoch 140/400
 - 0s - loss: 0.0478 - val_loss: 0.0810
Epoch 141/400
 - 0s - loss: 0.0466 - val_loss: 0.0821
Epoch 142/400
 - 0s - loss: 0.0472 - val_loss: 0.0837
Epoch 143/400
 - 0s - loss: 0.0476 - val_loss: 0.0834
Epoch 144/400
 - 0s - loss: 0.0474 - val_loss: 0.0828
Epoch 145/400
 - 0s - loss: 0.0462 - val_loss: 0.0794
Epoch 146/400
 - 0s - loss: 0.0445 - val_loss: 0.0838
Epoch 147/400
 - 0s - loss: 0.0451 - val_loss: 0.0854
Epoch 148/400
 - 0s - loss: 0.0456 - val_l

Epoch 282/400
 - 0s - loss: 0.0346 - val_loss: 0.0874
Epoch 283/400
 - 0s - loss: 0.0321 - val_loss: 0.0929
Epoch 284/400
 - 0s - loss: 0.0321 - val_loss: 0.0924
Epoch 285/400
 - 0s - loss: 0.0320 - val_loss: 0.0944
Epoch 286/400
 - 0s - loss: 0.0321 - val_loss: 0.0888
Epoch 287/400
 - 0s - loss: 0.0324 - val_loss: 0.0936
Epoch 288/400
 - 0s - loss: 0.0332 - val_loss: 0.0888
Epoch 289/400
 - 0s - loss: 0.0330 - val_loss: 0.0910
Epoch 290/400
 - 0s - loss: 0.0329 - val_loss: 0.0960
Epoch 291/400
 - 0s - loss: 0.0324 - val_loss: 0.0951
Epoch 292/400
 - 0s - loss: 0.0334 - val_loss: 0.0910
Epoch 293/400
 - 0s - loss: 0.0319 - val_loss: 0.0906
Epoch 294/400
 - 0s - loss: 0.0326 - val_loss: 0.0928
Epoch 295/400
 - 0s - loss: 0.0326 - val_loss: 0.0913
Epoch 296/400
 - 0s - loss: 0.0338 - val_loss: 0.0923
Epoch 297/400
 - 0s - loss: 0.0363 - val_loss: 0.0955
Epoch 298/400
 - 0s - loss: 0.0331 - val_loss: 0.0929
Epoch 299/400
 - 0s - loss: 0.0319 - val_loss: 0.0956
Epoch 300/400
 - 0s - loss: 

In [69]:
Base_output = base_model.predict(encoder_train)

In [149]:
def calc_distance(s1,s2):
    return(np.sqrt((s1[0] - s2[0])**2 + (s1[1] - s2[1])**2))

def get_nearest_data(s_train,s_test,y_train,k):
    dist_mat = np.zeros((len(s_test),len(s_train)))
    nearest_var = np.zeros((len(s_test),2*k))
    for i in range(len(s_train)):
        for j in range(len(s_test)):
            dist_mat[j][i] = calc_distance(s_train[i],s_test[j])
    for i in range(len(s_test)):
        a = dist_mat[i]
        if 0 in a:
            v = k+1
        b = np.argpartition(a,v)[:v]
        g = np.where(a == 0)
        b = np.delete(b, np.where(b == g[0][0]))
        y_val = []
        for index in b:
            y_val.append(y_train[index][0])
            y_val.append(y_train[index][1])
        nearest_var[i] = y_val
    return nearest_var
        
    
            

In [132]:
a = np.array([8,7,1.0,4,5])
g = np.where(a == 1)

b = np.array([1,2,3,4])
b = np.delete(b, np.where(b == g[0][0]))

In [163]:
nearest_var_testing = get_nearest_data(s_train,s_train,y_train,1)

In [151]:
final_input = np.column_stack((Base_output,nearest_var))

In [100]:
s_train[0:5]

array([[0.89873418, 0.88607595],
       [0.7721519 , 0.88607595],
       [0.11392405, 0.75949367],
       [0.62025316, 0.30379747],
       [0.39240506, 0.01265823]])

In [162]:
y_train

array([[-0.14710375,  0.10328696],
       [ 0.13410708,  0.57375816],
       [-0.99020328, -0.69902861],
       ...,
       [-0.16660055, -0.22403512],
       [-1.47078485, -1.12359922],
       [ 0.3786571 , -0.78806648]])

In [154]:
dist_mat = np.zeros((len(s_train),len(s_train)))

for i in range(len(s_train)):
    for j in range(len(s_train)):
        dist_mat[j][i] = calc_distance(s_train[i],s_train[j])


In [161]:
nearest_var_testing

array([[-0.26158856,  0.22170861, -0.24233602,  0.02296562, -0.29096899,
        -0.06318914],
       [ 0.60790409,  0.57331562, -0.79992057,  0.53085907, -0.1486718 ,
         0.48959046],
       [-0.95850801, -0.87930102, -0.11424227, -0.77654811, -0.44276466,
        -0.69072311],
       ...,
       [-0.80960683, -0.23544391,  0.12411374, -0.34868941, -0.54145416,
        -0.10209929],
       [-1.11402843, -1.25376512, -0.5925116 , -1.16859963, -1.13174626,
        -1.18079834],
       [-1.09777256, -0.79960374, -0.27403662, -0.73971068,  0.24625785,
        -0.62034769]])

In [159]:
## raf
c = np.delete(dist_mat, np.where(dist_mat == 0))
min(np.delete(dist_mat[0], np.where(dist_mat[0] == 0)))

0.012658227848100778

In [81]:
s_train[0:5]

array([[0.89873418, 0.88607595],
       [0.7721519 , 0.88607595],
       [0.11392405, 0.75949367],
       [0.62025316, 0.30379747],
       [0.39240506, 0.01265823]])

In [82]:
y_train[0:5]

array([[-0.14710375,  0.10328696],
       [ 0.13410708,  0.57375816],
       [-0.99020328, -0.69902861],
       [ 0.31269717, -0.70862401],
       [-0.01160224, -0.52233971]])

In [89]:
calc_distance(s_train[0],s_train[4])

1.0095680699539455

In [153]:
final_input


array([[-0.24832465,  0.03340811,  1.40403506, ...,  0.31626056,
        -0.17471588, -0.06502917],
       [ 0.26667354,  0.66589516,  0.33507061, ..., -0.65872623,
        -0.055674  , -0.03655024],
       [-0.52295393, -0.7851153 , -1.05970829, ..., -0.76934687,
         0.34550145, -0.92559299],
       ...,
       [-0.4663837 , -0.37715542, -0.41386661, ..., -0.10209929,
        -0.42496055, -0.58882242],
       [-0.64117116, -1.24829865, -1.11402843, ..., -1.18427565,
        -1.81753461, -1.01850024],
       [-0.59081292, -0.81809449, -1.09777256, ..., -0.57049535,
         0.93592028, -0.4463411 ]])

In [165]:
final_input_dim = Input(shape = (final_input.shape[1], ))
layer = Dense(2, kernel_initializer='he_uniform', activation = 'linear')(final_input_dim)
final_model = Model(inputs = final_input_dim, outputs = layer)
optimizer = keras.optimizers.Adam(lr=0.01)
# Compile the Model
final_model.compile(optimizer = optimizer, loss = 'mse')
final_model.summary()
final_model.fit(final_input, y_train,validation_split = 0.1, epochs = 300,
                batch_size = 128,verbose = 2)


Model: "model_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        (None, 22)                0         
_________________________________________________________________
dense_31 (Dense)             (None, 2)                 46        
Total params: 46
Trainable params: 46
Non-trainable params: 0
_________________________________________________________________
Train on 5184 samples, validate on 576 samples
Epoch 1/300
 - 0s - loss: 0.9567 - val_loss: 0.1793
Epoch 2/300
 - 0s - loss: 0.2684 - val_loss: 0.0884
Epoch 3/300
 - 0s - loss: 0.1256 - val_loss: 0.0825
Epoch 4/300
 - 0s - loss: 0.0772 - val_loss: 0.0858
Epoch 5/300
 - 0s - loss: 0.0556 - val_loss: 0.0874
Epoch 6/300
 - 0s - loss: 0.0441 - val_loss: 0.0875
Epoch 7/300
 - 0s - loss: 0.0376 - val_loss: 0.0893
Epoch 8/300
 - 0s - loss: 0.0336 - val_loss: 0.0903
Epoch 9/300
 - 0s - loss: 0.0311 - val_loss: 0.0911
Epoch 10/

Epoch 143/300
 - 0s - loss: 0.0270 - val_loss: 0.0946
Epoch 144/300
 - 0s - loss: 0.0270 - val_loss: 0.0963
Epoch 145/300
 - 0s - loss: 0.0270 - val_loss: 0.0933
Epoch 146/300
 - 0s - loss: 0.0271 - val_loss: 0.0946
Epoch 147/300
 - 0s - loss: 0.0271 - val_loss: 0.0938
Epoch 148/300
 - 0s - loss: 0.0275 - val_loss: 0.0930
Epoch 149/300
 - 0s - loss: 0.0276 - val_loss: 0.0939
Epoch 150/300
 - 0s - loss: 0.0273 - val_loss: 0.0944
Epoch 151/300
 - 0s - loss: 0.0272 - val_loss: 0.0933
Epoch 152/300
 - 0s - loss: 0.0272 - val_loss: 0.0941
Epoch 153/300
 - 0s - loss: 0.0274 - val_loss: 0.0946
Epoch 154/300
 - 0s - loss: 0.0272 - val_loss: 0.0949
Epoch 155/300
 - 0s - loss: 0.0272 - val_loss: 0.0927
Epoch 156/300
 - 0s - loss: 0.0272 - val_loss: 0.0955
Epoch 157/300
 - 0s - loss: 0.0272 - val_loss: 0.0954
Epoch 158/300
 - 0s - loss: 0.0274 - val_loss: 0.0948
Epoch 159/300
 - 0s - loss: 0.0272 - val_loss: 0.0947
Epoch 160/300
 - 0s - loss: 0.0271 - val_loss: 0.0943
Epoch 161/300
 - 0s - loss: 

Epoch 295/300
 - 0s - loss: 0.0272 - val_loss: 0.0924
Epoch 296/300
 - 0s - loss: 0.0272 - val_loss: 0.0960
Epoch 297/300
 - 0s - loss: 0.0272 - val_loss: 0.0941
Epoch 298/300
 - 0s - loss: 0.0271 - val_loss: 0.0945
Epoch 299/300
 - 0s - loss: 0.0271 - val_loss: 0.0953
Epoch 300/300
 - 0s - loss: 0.0274 - val_loss: 0.0950


In [58]:
base_output = base_model.predict(encoder_test)

ValueError: Graph disconnected: cannot obtain value for tensor Tensor("input_4:0", shape=(?, 120), dtype=float32) at layer "input_4". The following previous layers were accessed without issue: []